In [105]:
import re
import pandas as pd

In [106]:
data_origin = pd.read_csv("./sources/2017.csv",header=0)

In [110]:
def getlink(feed):
    print("现在正在处理",feed['school'])
    try:
        link = re.findall(r'href="(.*)"',feed['link'])[0]
    except TypeError:
        print(feed['school'],"文件链接异常")
        feed['getlink']='error'
        return feed
    except IndexError:
        print(feed['school'],"文件链接异常")
        feed['getlink']='error'
        return feed
    feed['getlink']=link
    return feed
data_origin['school'] = data_origin['school'].str.strip()
data_origin = data_origin.apply(getlink,axis=1)
data_origin.to_excel("./sources/2017link.xlsx")


现在正在处理 2017年北京大学毕业生就业质量年度报告
现在正在处理 2017年清华大学毕业生就业质量报告
现在正在处理 中国人民大学2017年毕业生就业质量报告
现在正在处理 北京师范大学
现在正在处理 中国农业大学2017年毕业生就业质量报告出炉
现在正在处理 北京外国语大学2017年毕业生就业质量报告出炉
现在正在处理 北京语言大学2017年毕业生就业质量年度报告
现在正在处理 北京科技大学2017年毕业生就业质量年报发布
现在正在处理 北京化工大学2017年毕业生就业质量报告出炉
现在正在处理 2017北京交通大学毕业生就业质量报告出炉
现在正在处理 北京邮电大学2017年毕业生就业质量报告
现在正在处理 中国地质大学（北京）2017届毕业生就业质量报告
现在正在处理 中国矿业大学2017年毕业生就业质量年度报告
现在正在处理 中国石油大学（北京）2017年毕业生就业质量报告发布
现在正在处理 北京林业大学2017年本科毕业生就业质量年度报告
现在正在处理 中国传媒大学2017届毕业生就业质量报告
现在正在处理 中央财经大学2017年毕业生就业质量报告
现在正在处理 中国政法大学
现在正在处理 中央音乐学院2017年毕业生就业质量报告
现在正在处理 中央戏剧学院
现在正在处理 中央美术学院2017届毕业生就业质量报告
现在正在处理 北京中医药大学2017届毕业生就业质量报告
现在正在处理 对外经济贸易大学2017年就业质量报告
现在正在处理 华北电力大学2017届毕业生就业质量年度报告
现在正在处理 南开大学2017届毕业生就业质量报告发布
现在正在处理 天津大学2017年就业质量报告发布
现在正在处理 大连理工大学2017届毕业生就业质量报告
现在正在处理 东北大学就业报告 信息传输行业就业比例快速增长
现在正在处理 吉林大学2017届毕业生就业质量年度报告
现在正在处理 东北师范大学2017年毕业生就业质量年度报告出炉
东北师范大学2017年毕业生就业质量年度报告出炉 文件链接异常
现在正在处理 东北林业大学2017年毕业生就业质量报告出炉
东北林业大学2017年毕业生就业质量报告出炉 文件链接异常
现在正在处理 复旦大学2017
现在正在处理 上海交通大学2017届毕业生就业质量报告
现在正在处理 同济大学2017年毕业生就业质量报告
现在正在处理 华

### Download Files

In [2]:
#download_file
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
from tqdm import tqdm
import re
import chardet
from selenium import webdriver

def download_file(school,link):
    print("Ready to retrieve files of ",school,"\n")
    try:
        response = requests.get(link,stream=True)
        content_size = int(response.headers['Content-Length'])/1024
    except:
        print("When downloading",school,",an error occurred.")
        return
    if os.path.isdir("./downloads/2018") is False:
        os.mkdir("./downloads/2018")
    path = "./downloads/2018/"+school+".pdf"
    with open(path,'wb') as f:
        for i in tqdm(
            iterable = response.iter_content(1024),
            total=content_size,
            unit='k',
            desc=school
        ):
            f.write(i)
    return 0

def read_link(school='test',link='none'):
    print("Trying to get response from ",school,"\n")
    ua = {'User-Agent':'Mozilla/5.0 (Windows NT 6.2; Win64; x64) Chrome/65.0.3325.181'}
    try:
        rqs=requests.get(link,headers=ua,timeout=3.0)
        rqs.encoding = chardet.detect(rqs.content)['encoding'] #requests请求过程
        if rqs.status_code==404:
            print(school,"网页404。")
            return -1
    except(Exception) as e:
        print(school,"网页请求异常。")
        print(e)
        return -1
    html = rqs.content.decode('utf-8')
    soup = bs(html,'lxml')
    print(soup.find_all("title"))
    print(soup.find_all(name='a',attrs={"href":re.compile(r'.*')}))


        

In [ ]:
#2018
data = pd.read_excel("./sources/2018link.xlsx",header=0)
df_school_link = data[['school','getlink']]
for index,item in df_school_link.iterrows():
    if item['getlink'] =='error':
        print(item['school'],'下载链接异常')
        continue
    download_file(item['school'],item['getlink'])

In [3]:
#2019
data = pd.read_excel("./sources/links.xlsx")
data['status'] = 'Ready'
print(data.head(5))


   school                                              links status
0    北京大学  https://scc.pku.edu.cn/news_ff8080816f1344ed01...  Ready
1  中国人民大学  http://rdjy.ruc.edu.cn/html/main/col12/2019-12...  Ready
2    清华大学  http://career.tsinghua.edu.cn/publish/career/8...  Ready
3  北京科技大学  http://job.ustb.edu.cn/front/showContent.jspa?...  Ready
4  北京化工大学   http://www.job.buct.edu.cn/news.asp?NewsId=38396  Ready


In [6]:
link = "https://www.cau.edu.cn/art/2019/12/31/art_10353_657700.html"
read_link(school="人大",link=link)
driver = webdriver.Edge()
driver.get(link)

Trying to get response from  人大 

[<title>中国农业大学 学校公告 中国农业大学2019年毕业生就业质量报告</title>]
[<a href="http://my.cau.edu.cn/" target="_blank">校内平台</a>, <a href="https://vpn.cau.edu.cn" target="_blank">校外VPN</a>, <a href="https://mail.cau.edu.cn" target="_blank">网络邮局</a>, <a href="/col/col10235/index.html">考生</a>, <a href="http://110th.cau.edu.cn/" target="_blank">校友</a>, <a href="/col/col10243/index.html">访客</a>, <a href="/col/col36015/index.html" target="_blank">应聘者</a>, <a href="http://gzc.cau.edu.cn/sfw_cms/e?page=cms.index" target="_blank">工程招标</a>, <a href="http://my.cau.edu.cn/" target="_blank">校内平台</a>, <a href="https://vpn.cau.edu.cn" target="_blank">校外VPN</a>, <a href="https://mail.cau.edu.cn" target="_blank">网络邮局</a>, <a href="/col/col10235/index.html">考生</a>, <a href="http://110th.cau.edu.cn/" target="_blank">校友</a>, <a href="/col/col10243/index.html">访客</a>, <a href="/col/col36015/index.html" target="_blank">应聘者</a>, <a href="http://gzc.cau.edu.cn/sfw_cms/e?page=cms.index" target="_

SessionNotCreatedException: Message: session not created: This version of Microsoft Edge WebDriver only supports Microsoft Edge version 105
Current browser version is 103.0.1264.44 with binary path C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe
